In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

### Etape 1:
1/ Importation des données.
2/ "URL", "RESPONSE", "HTML" permettent de récupérer le texte du site.
3/ "SOUP" permet d'utiliser les données pour les scraper.

In [2]:
url = "https://www.vlr.gg/matches/results"
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
data = soup.find_all('div')

### Etape 2:
1/ Création du dataframe "df1" recueillant toutes les données.
1.1/ "a" est le dataframe pour la phase de la compétition.
1.2/ "b" est le dataframe pour la compétition.
2/ Les deux dataframes "a" et "b" sont fusionnés dans le df1.

In [3]:
df1 = pd.DataFrame()

a = pd.DataFrame(columns=["1"])
b = pd.DataFrame(columns=["2"])

col = soup.find_all('div', class_='match-item-event-series text-of')
for cols in col:
    texts = cols.find_all(string=True, recursive=False)
    text1 = texts[0].strip()
    a = a._append({'1':text1}, ignore_index=True)
col1 = soup.find_all('div', class_='match-item-event text-of')
for cols in col1:
    texts1 = cols.find_all(string=True, recursive=False)
    text2 = texts1[1].strip()
    b = b._append({'2':text2}, ignore_index=True)

df1 = b.join(a, how='inner')
df1

,2,1
0,Knights Monthly Gauntlet 2023: October,Playoffs–Semifinals
1,Knights Monthly Gauntlet 2023: October,Playoffs–Semifinals
2,Multiplatform Esports Game 2023,Playoffs–Lower Final
3,VALORANT Challengers 2023 DACH: Arcade,Group Stage–Week 6
4,VALORANT Challengers 2023 DACH: Arcade,Group Stage–Week 6
5,China Evolution Series Act 2: Selection,Group Stage–Winner's (B)
6,China Evolution Series Act 2: Selection,Group Stage–Winner's (A)
7,Predator League Japan 2024,Main Event–Grand Final
8,Liquid Open 2023: Oceania,Playoffs–Grand Final
9,Predator League Japan 2024,Main Event–Consolation Final


In [12]:
df2 = pd.DataFrame()
c = pd.DataFrame(columns=["3"])
d = pd.DataFrame(columns=["4"])
col = soup.find_all('div', class_='match-item-vs-team-name')
for cols in col:
    texts = cols.find_all(string=True, recursive=True)
    text = texts[2].strip()
    text1 = texts[3].strip()
    c = c._append({'3':text}, ignore_index=True)
    d = d._append({'4':text1}, ignore_index=True)

c = c.replace('', np.nan, regex=True)
c = c.dropna()
c = c.reset_index(drop=True)
d = d.replace('', np.nan, regex=True)
d = d.dropna()
d = d.reset_index(drop=True)

df2 = d.join(c, how='inner')
df1 = df1.join(df2, how='inner')

In [13]:
df1

,2,1,4,3
0,Knights Monthly Gauntlet 2023: October,Playoffs–Semifinals,Argon Black,Square Nation
1,Knights Monthly Gauntlet 2023: October,Playoffs–Semifinals,Turtle Troop,Version1
2,Multiplatform Esports Game 2023,Playoffs–Lower Final,XIT,MIBR
3,VALORANT Challengers 2023 DACH: Arcade,Group Stage–Week 6,CGN Esports,Reveal Multigaming
4,VALORANT Challengers 2023 DACH: Arcade,Group Stage–Week 6,ALTERNATE aTTaX,AEX-1
5,China Evolution Series Act 2: Selection,Group Stage–Winner's (B),Attacking Soul Esports,Rare Atom
6,China Evolution Series Act 2: Selection,Group Stage–Winner's (A),EDward Gaming,Bilibili Gaming
7,Predator League Japan 2024,Main Event–Grand Final,FAV gaming,CREST GAMING Zst
8,Liquid Open 2023: Oceania,Playoffs–Grand Final,Project X,Hanma
9,Predator League Japan 2024,Main Event–Consolation Final,REJECT,Sengoku Gaming
